<a href="https://colab.research.google.com/github/frank22004/GenAI/blob/main/GenAI%E6%9C%9F%E6%9C%AB%E5%B0%88%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.1 安裝套件

In [ ]:
!pip install -U langchain langchain-community pypdf python-docx sentence-transformers faiss-cpu gradio openai wordcloud dash fuss fuzzywuzzy

安裝中文字體

In [ ]:
!wget -O TaipeiSansTCBeta-Regular.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download

In [ ]:
# 載入必要套件
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 文字雲
from wordcloud import WordCloud

import os
from openai import OpenAI
import gradio as gr

# 互動設計用
from ipywidgets import interact_manual

from google.colab import userdata

# 文件分析
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import Document

# 模糊比對
from fuzzywuzzy import fuzz, process

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

class CustomE5Embedding(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        return super().embed_query(f"query: {text}")

## 1.2. 設定LLM

  ### 1.2.1 設定API來自Llama 3-70b

In [ ]:
api_key = userdata.get('Groq')
os.environ["OPENAI_API_KEY"] = api_key
model = "llama3-70b-8192"
base_url="https://api.groq.com/openai/v1"

### 1.2.2 設定OpenAI跟Prompt

In [ ]:
client = OpenAI(
    base_url=base_url
)

system_prompt = "你是臨床心理與照護的專家，請根據患者書寫的日記。請分析當下情緒，並給予適當鼓勵。請務必用台灣的中文回應。"

1.2.3. Test

In [ ]:
prompt = "今天的下午改變了我整個人生。醫生告訴我：「是乳癌，第二期。」我愣住了，什麼話都說不出來。那一瞬間，我的心跳變得好快，耳邊像嗡嗡作響，世界彷彿靜止。怎麼可能？我才三十八歲，身體一向健康，也沒有家族病史，怎麼會是我？我一直在想，是不是報告弄錯了？是不是只是良性腫瘤？但醫生的眼神很肯定，沒有空間懷疑。"


messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt}
]

In [ ]:
response = client.chat.completions.create(
    model=model,
    messages=messages
    )

print(response.choices[0].message.content)

## 1.3 讀取NTUSD中文正負向詞彙字典

台灣大學自然語言處理實驗室(NTU Natural Language Processing Laboratory)[陳信希教授團隊](https://nlg.csie.ntu.edu.tw/advisor.php)，利用Jieba斷詞和Naive Bayes分類器所建立中文情緒辭典(NTUSD)，其涵蓋8277個負面情緒詞跟2810個正面情緒詞。

故下載Github的NTUSD專案

In [ ]:
!git clone https://github.com/sweslo17/chinese_sentiment.git

### 1.3.1. 將txt轉成List

In [ ]:
postiveEmo = TextLoader('/content/chinese_sentiment/dict/ntusd-positive.txt')
negativeEmo = TextLoader('/content/chinese_sentiment/dict/ntusd-negative.txt')

In [ ]:
postiveEmo = postiveEmo.load()
negativeEmo = negativeEmo.load()

In [ ]:
postiveEmo = postiveEmo[0].page_content.split('\n')
negativeEmo = negativeEmo[0].page_content.split('\n')

In [ ]:
postiveEmo = [i.split('\t')[0] for i in postiveEmo]
negativeEmo = [i.split('\t')[0] for i in negativeEmo]

#### 顯示正面情緒詞

In [ ]:
def print_pos(n=0):
    print(postiveEmo[n])

interact_manual(print_pos,n=(0,len(postiveEmo)-1))

#### 顯示負面情緒詞

In [ ]:
def print_neg(n=0):
    print(negativeEmo[n])

interact_manual(print_neg,n=(0,len(negativeEmo)-1))

### 1.3.2. 建立正面情緒向量資料庫

In [ ]:
embedding_model = CustomE5Embedding(model_name="intfloat/multilingual-e5-small")

# Create Document objects from the list of positive emotion strings
positive_documents = [Document(page_content=emo) for emo in postiveEmo]

# Pass the list of Document objects to FAISS.from_documents
vectorstore_pos = FAISS.from_documents(positive_documents, embedding_model)

嘗試RAG

In [ ]:
retriever_pos = vectorstore_pos.as_retriever()

AI agent回答的內容

In [ ]:
docs = retriever_pos.get_relevant_documents(response.choices[0].message.content)
docs

模擬的患者日記

In [ ]:
docs_pa = retriever_pos.get_relevant_documents(prompt)
docs_pa

### 1.3.3 建立負面情緒向量資料庫

In [ ]:
# Create Document objects from the list of positive emotion strings
negative_documents = [Document(page_content=emo) for emo in negativeEmo]

# Pass the list of Document objects to FAISS.from_documents
vectorstore_neg = FAISS.from_documents(negative_documents, embedding_model)

嘗試RAG

In [ ]:
retriever_neg = vectorstore_neg.as_retriever()

AI agent的回答

In [ ]:
doc_neg = retriever_neg.get_relevant_documents(response.choices[0].message.content)
doc_neg

模擬的患者日記

In [ ]:
doc_neg_pa = retriever_neg.get_relevant_documents(prompt)
doc_neg_pa

## 1.4. 分析情緒

### 1.4.1. 正面情緒

設定Prompt

In [ ]:
customized_prompt = """
根據下列情緒字詞：
{retrieved_chunks}

分析患者的日記：{question}

請回覆日記所載內容是否有符合所列情緒字詞，
若有請回答「本日記符合以下的情緒」，並列出符合情緒字詞，
若沒有符合的情緒，請直接回答「沒有符合的情緒」。
"""

In [ ]:
retrieved_pos_chunks = "\n\n".join([doc.page_content for doc in docs_pa])

# 將自定 prompt 套入格式
final_prompt = customized_prompt.format(retrieved_chunks=retrieved_pos_chunks, question=prompt)

# 呼叫 OpenAI API
response_pos_ana = client.chat.completions.create(
  model=model,
  messages=[
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": final_prompt},
  ]
)

answer_pos_ana = response_pos_ana.choices[0].message.content
print(answer_pos_ana)

### 1.4.2. 負面情緒

In [ ]:
retrieved_neg_chunks = "\n\n".join([doc.page_content for doc in doc_neg_pa])

# 將自定 prompt 套入格式
final_prompt_neg = customized_prompt.format(retrieved_chunks=retrieved_neg_chunks, question=prompt)

# 呼叫 OpenAI API
response_neg_ana = client.chat.completions.create(
  model=model,
  messages=[
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": final_prompt_neg},
  ]
)

print(response_neg_ana.choices[0].message.content)

### 1.4.3. 綜合正負面情緒分析

In [ ]:
mixed_prompt = """
根據下列正面情緒字詞：
{retrieved_chunks_post}

及下列負面情緒字詞:
{retrieved_chunks_neg}

分析患者的日記：{question}

請回覆日記所載內容是否所有符合所列情緒字詞，並繪製情緒雲，
若沒有符合的情緒，請直接回答「沒有符合的情緒」。
"""

In [ ]:
# 將自定 prompt 套入格式
final_prompt = mixed_prompt.format(retrieved_chunks_post=retrieved_pos_chunks, retrieved_chunks_neg=retrieved_neg_chunks, question=prompt)

# 呼叫 OpenAI API
response_ana = client.chat.completions.create(
  model=model,
  messages=[
      {"role": "system", "content": mixed_prompt},
      {"role": "user", "content": final_prompt},
  ]
)

print(response_ana.choices[0].message.content)

不用辭典，直接請Agent分析情緒

In [ ]:
sys_prompt = """
分析患者的日記：{question}，所載內容中所有的情緒字詞及出現次數。
以表格方式回答各個情緒字詞及出現次數，
不要有任何其他文字或句子
也不要有標題欄位
須以繁體中文回覆
"""

In [ ]:
# 將自定 prompt 套入格式
final_wo_prompt = sys_prompt.format(question=prompt)

# 呼叫 OpenAI API
response_wo_ana = client.chat.completions.create(
  model=model,
  messages=[
      {"role": "system", "content": sys_prompt},
      {"role": "user", "content": final_wo_prompt},
  ]
)

print(response_wo_ana.choices[0].message.content)

In [ ]:
emorespon = response_wo_ana.choices[0].message.content.split('\n')
emorespon[0].split(' ')
emopatient ={ }

for i in range(len(emorespon)):
  key, value = emorespon[i].split(' ')
  emopatient.update({key: int(value)})

print(emopatient)

### 1.4.4. 文字雲

詢問Gmini如何繪製文字雲


*   Prompt: 「給我一段python產生文字雲」
*   Gemini:


> 安裝套件： 首先使用 !pip install wordcloud==1.8.2.2 安裝 wordcloud 套件。
>
> 匯入套件： 匯入 WordCloud 和 matplotlib.pyplot。
>
> 準備文字資料： 將您想要產生文字雲的文字放入 text 變數中。
>
> 建立 WordCloud 物件： 使用 WordCloud() 建立一個文字雲物件。
>
>  font_path='SimHei.ttf'：這是一個重要的參數，用於指定文字雲的字體。如果您處理的文字包含中文，您需要指定一個支援中文的字體檔案。在 Google Colab 中，通常可以使用內建的字體，或者您可以上傳自己的字體檔案。這裡使用了一個常見的中文字體範例。
>
>  background_color='white'：設定文字雲的背景顏色。您可以根據需要更改。
>
> 產生文字雲： 使用 .generate(text) 方法根據您的文字資料產生文字雲。
>
> 顯示文字雲： 使用 matplotlib.pyplot 來顯示產生的文字雲。
>
> plt.figure(figsize=(10, 8))：設定圖形的大小。
>
> plt.imshow(wordcloud, interpolation='bilinear')：顯示文字雲圖像。
>
> plt.axis('off')：關閉座標軸。
>
> plt.show()：顯示圖形。
>
> 注意事項：
>
> 中文字體： 如果您的文字包含中文，請確保您指定的 font_path 指向一個有效的、支援中文的字體檔案。您可能需要上傳字體檔案到您的 Colab 環境中。
>
> 文字預處理： 在實際應用中，您可能需要對文字進行預處理，例如去除標點符號、停用詞等，以獲得更好的文字雲效果。
> 將這段程式碼複製到您的 Google Colab 筆記本中執行，就可以產生文字雲了。



In [ ]:
# 設定 WordCloud 參數，例如字體、背景顏色等
# 如果需要顯示中文，請指定中文字體
# 參照https://colab.research.google.com/github/willismax/matplotlib_show_chinese_in_colab/blob/master/matplotlib_show_chinese_in_colab.ipynb#scrollTo=AByB62T315mR
# 使用TaipeiSans
wordcloud = WordCloud(font_path='TaipeiSansTCBeta-Regular.ttf',
                      background_color='white').generate(response_wo_ana.choices[0].message.content)

# 顯示文字雲
plt.figure(figsize=(10, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

依照文字出現頻率而繪製的文字雲

In [ ]:
# 從頻率字典產生文字雲
wordcloud_fre = WordCloud(font_path='TaipeiSansTCBeta-Regular.ttf',
                      background_color='white')
wordcloud_fre.generate_from_frequencies(emopatient)

# 顯示文字雲
plt.figure(figsize=(10, 8))
plt.imshow(wordcloud_fre, interpolation='bilinear')
plt.axis('off')
plt.show()

## 1.5. 情緒分類

使用NTUSD辭典，發現並不能有效區別日記中的情緒，故採用皮克斯電影中「腦筋急轉彎」中，情緒角色，作為分類的依據

In [ ]:
emotion_dict = {
    "快樂": [
        "開心", "快樂", "高興", "喜悅", "幸福", "滿足", "爽快", "愉快", "輕鬆", "欣慰",
        "舒服", "激動", "興奮", "雀躍", "開懷", "如釋重負", "安心", "欣喜", "甜蜜", "幸福感"
    ],
    "焦慮": [
        "焦慮", "不安", "緊張", "擔心", "煩躁", "惶恐", "坐立難安", "心神不寧", "心慌",
        "忐忑", "惴惴", "沒安全感", "提心吊膽", "急躁", "煩悶", "無措", "亂", "懷疑", "恐慌", "心煩"
    ],
    "討厭": [
        "討厭", "厭惡", "排斥", "嫌棄", "反感", "膩", "煩", "受不了", "厭煩", "嫌",
        "刺眼", "刺耳", "懶得理", "懶得看", "不想理", "懷疑", "抗拒", "否定", "看不順眼", "不耐煩"
    ],
    "憤怒": [
        "生氣", "憤怒", "怒火", "氣憤", "氣死", "發火", "火大", "抓狂", "咬牙", "怒氣沖天",
        "怒目", "怒吼", "冒火", "氣炸", "惱怒", "不爽", "發飆", "氣到哭", "氣到笑", "動怒"
    ],
    "憂鬱": [
        "憂鬱", "悲傷", "難過", "沮喪", "失落", "空虛", "低落", "無助", "孤單", "痛苦",
        "不想活", "想哭", "沉重", "壓抑", "疲憊", "崩潰", "麻木", "悶", "失望", "黑暗"
    ],
    "驚恐": [
        "害怕", "恐懼", "驚嚇", "驚恐", "心驚膽跳", "受驚", "緊張兮兮", "慌張", "不知所措", "發抖",
        "恐慌", "戰慄", "魂飛魄散", "慌亂", "怕死", "不寒而慄", "倒抽一口氣", "心跳加速", "冒冷汗", "驚慌失措"
    ],
    "羨慕": [
        "羨慕", "嫉妒", "眼紅", "酸", "忌妒", "心酸", "想要", "羨慕不已", "好羨慕", "羨慕得要命",
        "羨慕他人", "羨慕自己沒有", "怨", "羨望", "眼饞", "欣羨", "仰慕", "欽佩", "欽羨", "失落地看著"
    ],
    "無所謂": [
        "無所謂", "隨便", "沒差", "無感", "無聊", "懶得管", "不在意", "冷淡", "不關心", "冷漠",
        "看開", "不放在心上", "心如止水", "不在乎", "覺得無趣", "沒興趣", "被動", "沒感覺", "已經習慣", "沒什麼好說"
    ],
    "害羞": [
        "害羞", "不好意思", "臉紅", "怕見人", "低頭", "不敢說", "不敢看", "緊張", "羞澀", "羞愧",
        "怕被笑", "躲避", "閃躲", "遮掩", "不自在", "小聲說", "結巴", "慌張", "手足無措", "拘謹"
    ]
}

In [ ]:
type(emotion_dict)
emotion_dict.get("羨慕",0)

分割情緒字典的dictionary

In [ ]:
items = emotion_dict.items()

for key, value in items:
  emo = key
  print(key, value)
#  print(emo)

測試模糊比對

In [ ]:
process.extract("驚恐", emotion_dict.get("驚恐",0))

### 1.5.1. 分類AI agent的情緒

In [ ]:
sum = 0
emo_list ={}
emo_frame = pd.DataFrame()

# 讀取AI agent分析的所有字詞
for key, value in emopatient.items():
  # 依據情緒辭典的7個類別
  for emo in emotion_dict.keys():
    # 依依模糊比對
    results = process.extract(key, emotion_dict.get(emo,0))
#    print(results)
    # 加總該詞組內，最相近的4個詞的相似性總分
    for result in results:
      sum = sum + result[1]
#    print(sum)
    # 輸出成dict
    emo_list.update({emo: sum})
    sum = 0
#  print(emo_list)
  # 將dict轉成dataframe
  emo_frame = pd.concat([emo_frame, pd.DataFrame([emo_list])], ignore_index=True)
  emo_list = {}
print(emo_frame)

### 1.5.2. 加總所有情緒詞在9個類別的總分

In [ ]:
emotion = emo_frame.sum().sort_values(ascending=False)
print(emotion)

### 1.5.3. 繪製9個情緒類別的文字雲

In [ ]:
# 從頻率字典產生文字雲
wordcloud_tot = WordCloud(font_path='TaipeiSansTCBeta-Regular.ttf',
                      background_color='white')
wordcloud_tot.generate_from_frequencies(emotion)

# 顯示文字雲
plt.figure(figsize=(7, 5))
plt.imshow(wordcloud_tot, interpolation='bilinear')
plt.axis('off')
plt.show()

## 1.6. 建立Web

建立AI agent

In [ ]:
def chat_with_rag(user_input):
    # 將自定 prompt 套入格式
    final_prompt = mixed_prompt.format(retrieved_chunks_post=retrieved_pos_chunks,
                                       retrieved_chunks_neg=retrieved_neg_chunks,
                                       question=user_input)

    # 呼叫 OpenAI API
    response_ana = client.chat.completions.create(
      model=model,
      messages=[
        {"role": "system", "content": mixed_prompt},
        {"role": "user", "content": final_prompt},
      ]
    )

    chat_history.append((user_input, answer))
    return answer